In [1]:
!pip install opencv_python

^C


In [7]:
# converting from pdf format to jpg format
!pip install mtcnn

In [1]:
import fitz  # PyMuPDF

def extract_images_from_pdf(pdf_path):
    pdf_document = fitz.open(pdf_path)
    
    for page_num in range(len(pdf_document)):
        page = pdf_document[page_num]
        image_list = page.get_images(full=True)  # Get the images on the page
        
        for img_index, img in enumerate(image_list):
            xref = img[0]  # The xref number of the image
            base_image = pdf_document.extract_image(xref)  # Extract the image
            image_bytes = base_image["image"]  # Get the image bytes
            image_extension = base_image["ext"]  # Get the image extension
            
            # Save the image
            image_filename = f"extracted_image_page_{page_num+1}_img_{img_index+1}.{image_extension}"
            with open(image_filename, "wb") as img_file:
                img_file.write(image_bytes)
                print(f"Saved {image_filename}")
    
    pdf_document.close()

# Usage
extract_images_from_pdf("Pan.pdf")


Saved extracted_image_page_1_img_1.jpeg


In [2]:
import cv2
import numpy as np
import random

def preprocess_document(image_path):
    # Read the image
    image = cv2.imread(image_path, cv2.IMREAD_COLOR)

    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Enhance contrast using CLAHE
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    gray = clahe.apply(gray)

    # Apply Gaussian blur to reduce noise
    gray = cv2.GaussianBlur(gray, (5, 5), 0)

    # Darken the grayscale image
    dark_gray = cv2.addWeighted(gray, 0.5, np.zeros(gray.shape, gray.dtype), 0, 0)  # Reduces brightness

    # Unsharp Masking
    # Create a Gaussian kernel
    gaussian_blur = cv2.GaussianBlur(dark_gray, (5, 5), 0)
    # Subtract the Gaussian blur from the original image to enhance edges
    unsharp_masked = cv2.addWeighted(dark_gray, 2.5, gaussian_blur, -0.5, 0)  # 1.5 is the weight for sharpening

    # Use Canny edge detection
    edges = cv2.Canny(unsharp_masked, 50, 150)

    # Optional: Resize image to standard size
    processed_image = cv2.resize(unsharp_masked, (600, 400))

    return processed_image

def augment_image(image):
    # Random rotation
    angle = random.uniform(-15, 15)  # Rotate between -15 and 15 degrees
    height, width = image.shape[:2]
    center = (width // 2, height // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated_image = cv2.warpAffine(image, M, (width, height))

    # Random scaling
    scale = random.uniform(0.9, 1.1)  # Scale between 90% and 110%
    scaled_image = cv2.resize(rotated_image, None, fx=scale, fy=scale)

    return scaled_image



In [4]:

def main(image_path, output_path):
    # Preprocess the document
    processed_image = preprocess_document(image_path)

    # Augment the image
    #augmented_image = augment_image(processed_image)#

    # Save the final image
    cv2.imwrite(output_path, processed_image)
    print(f"Processed image saved to {output_path}")

# Example usage
image_path = "pusu.jpg"
output_path = "processed_pusu_image.jpg"
main(image_path, output_path)

Processed image saved to processed_pusu_image.jpg


In [ ]:
import cv2
from mtcnn import MTCNN

def detect_face(image):
    """
    Detect faces in an image using MTCNN.
    """
    detector = MTCNN()
    faces = detector.detect_faces(image)
    return faces

def extract_face(image, face):
    """
    Extract the face from the image based on the bounding box provided by MTCNN.
    """
    x, y, width, height = face['box']  # Extract the bounding box
    # Ensure the bounding box is within image dimensions
    x = max(0, x)
    y = max(0, y)
    width = min(width, image.shape[1] - x)
    height = min(height, image.shape[0] - y)
    extracted_face = image[y:y + height, x:x + width]
    return extracted_face

def rotate_image(image, angle):
    """
    Rotate the image by the specified angle.
    """
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)

    # Perform the rotation
    matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, matrix, (w, h))
    return rotated

# Main workflow
image_path = 'processed_pusu_image.jpg'  # Replace with your preprocessed image path
image = cv2.imread(image_path)

# Convert the image to RGB
rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Attempt to detect faces
faces = detect_face(rgb_image)

# Define the angles to try if no faces are detected
rotation_angles = [0,90, 180, 270 ,0, 15, 30, 45, 60, 75, 90, 105, 120, 135, 150, 165, 180 , 195, 210, 225, 240, 255, 270, 285, 300, 315,330]

# Rotate the image if no faces are detected
for angle in rotation_angles:
    if faces:  # Break the loop if faces are found
        break
    print(f"No face detected. Rotating the image by {angle} degrees.")
    rgb_image = rotate_image(rgb_image, angle)
    faces = detect_face(rgb_image)

# Process the detected face
if faces:
    # Extract the first detected face
    extracted_face = extract_face(rgb_image, faces[0])

    # Save the extracted face
    output_path = 'extracted_face_pusu.jpg'
    cv2.imwrite(output_path, cv2.cvtColor(extracted_face, cv2.COLOR_RGB2BGR))  # Save as BGR for OpenCV
    print(f"Face extracted and saved at {output_path}.")

    # Optionally display the extracted face
    cv2.imshow('Extracted Face', extracted_face)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
else:
    print("No faces detected in the image after all rotations.")
